In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler

In [2]:
dtr = pd.read_csv('diabetes.csv')

In [3]:
dtr.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
dtr.shape

(768, 9)

In [5]:
x=dtr.iloc[:,:-1].values
y=dtr.iloc[:,-1].values


In [6]:
scaler = MinMaxScaler()
x = scaler.fit_transform(x)
print(x)
print(y)

[[0.35294118 0.74371859 0.59016393 ... 0.50074516 0.23441503 0.48333333]
 [0.05882353 0.42713568 0.54098361 ... 0.39642325 0.11656704 0.16666667]
 [0.47058824 0.91959799 0.52459016 ... 0.34724292 0.25362938 0.18333333]
 ...
 [0.29411765 0.6080402  0.59016393 ... 0.390462   0.07130658 0.15      ]
 [0.05882353 0.63316583 0.49180328 ... 0.4485842  0.11571307 0.43333333]
 [0.05882353 0.46733668 0.57377049 ... 0.45305514 0.10119556 0.03333333]]
[1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0
 1 1 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1
 1 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0
 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1
 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0
 1 0 1 0 0 1 0 1 0 0 1 1 0 0 0 

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,random_state = 120)


In [8]:
x_train = X_train.T
y_train = y_train.reshape(1, y_train.shape[0])
x_test = X_test.T
y_test = y_test.reshape(1, y_test.shape[0])

In [9]:
input_layer=x_train.shape[0]
hidden_layer=12
output_layer=1


In [10]:
np.random.seed(1) 
W1 = np.random.uniform(-1,1,size=(hidden_layer, input_layer))
b1 = np.zeros((hidden_layer, 1))
W2 = np.random.uniform(-1,1,size=(output_layer, hidden_layer))
b2 = np.zeros((output_layer, 1))

In [11]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [12]:
def forward_propagation(W1,x_train,b1,W2,b2):
    
    Z1 = np.dot(W1, x_train) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    cache = {"Z1": Z1,"A1": A1,"Z2": Z2,"A2": A2}
    return A2, cache

In [13]:
def cost_main(A2,y_train):
    
    m = y_train.shape[1] 
    cost= np.multiply(np.log(A2), y_train) + np.multiply((1-y_train), np.log(1 - A2))
    cost = - np.sum(cost) / m
    cost = float(np.squeeze(cost))
    return cost
                   

In [14]:
def backward_propagation(W1,W2, cache, x_train, y_train):
    m = x_train.shape[1]
    A1 = cache['A1']
    A2 = cache['A2']
   
    dZ2 = A2-y_train
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1/m) * np.dot(dZ1, x_train.T) 
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2,"db2": db2}
    
    return grads


In [19]:
def gradient_descent(W1,b1,W2,b2, grads, learning_rate):
   
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    
    return W1,W2,b1,b2

In [20]:
def neural_network_model(x_train, y_train,num_iterations,W1,W2,b1,b2,learning_rate):
    
    for i in range(0, num_iterations):
        A2, cache = forward_propagation(W1,x_train,b1,W2,b2)
        cost =cost_main(A2,y_train)
        grads = backward_propagation(W1,W2, cache, x_train, y_train)
        W1,W2,b1,b2=gradient_descent(W1,b1,W2,b2, grads, learning_rate)
        if i % 5 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    return W1,W2,b1,b2

In [21]:
W1,W2,b1,b2=neural_network_model(x_train, y_train,3000,W1,W2,b1,b2,0.01)

Cost after iteration 0: 0.907554
Cost after iteration 5: 0.881852
Cost after iteration 10: 0.858526
Cost after iteration 15: 0.837497
Cost after iteration 20: 0.818658
Cost after iteration 25: 0.801880
Cost after iteration 30: 0.787017
Cost after iteration 35: 0.773911
Cost after iteration 40: 0.762401
Cost after iteration 45: 0.752327
Cost after iteration 50: 0.743532
Cost after iteration 55: 0.735867
Cost after iteration 60: 0.729195
Cost after iteration 65: 0.723387
Cost after iteration 70: 0.718328
Cost after iteration 75: 0.713915
Cost after iteration 80: 0.710056
Cost after iteration 85: 0.706671
Cost after iteration 90: 0.703691
Cost after iteration 95: 0.701053
Cost after iteration 100: 0.698706
Cost after iteration 105: 0.696605
Cost after iteration 110: 0.694713
Cost after iteration 115: 0.692996
Cost after iteration 120: 0.691427
Cost after iteration 125: 0.689983
Cost after iteration 130: 0.688645
Cost after iteration 135: 0.687397
Cost after iteration 140: 0.686223
Cost af

Cost after iteration 1195: 0.588084
Cost after iteration 1200: 0.587795
Cost after iteration 1205: 0.587507
Cost after iteration 1210: 0.587220
Cost after iteration 1215: 0.586933
Cost after iteration 1220: 0.586648
Cost after iteration 1225: 0.586363
Cost after iteration 1230: 0.586079
Cost after iteration 1235: 0.585795
Cost after iteration 1240: 0.585512
Cost after iteration 1245: 0.585230
Cost after iteration 1250: 0.584949
Cost after iteration 1255: 0.584668
Cost after iteration 1260: 0.584388
Cost after iteration 1265: 0.584109
Cost after iteration 1270: 0.583831
Cost after iteration 1275: 0.583553
Cost after iteration 1280: 0.583276
Cost after iteration 1285: 0.583000
Cost after iteration 1290: 0.582724
Cost after iteration 1295: 0.582449
Cost after iteration 1300: 0.582175
Cost after iteration 1305: 0.581901
Cost after iteration 1310: 0.581628
Cost after iteration 1315: 0.581356
Cost after iteration 1320: 0.581084
Cost after iteration 1325: 0.580813
Cost after iteration 1330: 0

Cost after iteration 2370: 0.536189
Cost after iteration 2375: 0.536024
Cost after iteration 2380: 0.535860
Cost after iteration 2385: 0.535696
Cost after iteration 2390: 0.535533
Cost after iteration 2395: 0.535369
Cost after iteration 2400: 0.535207
Cost after iteration 2405: 0.535045
Cost after iteration 2410: 0.534883
Cost after iteration 2415: 0.534721
Cost after iteration 2420: 0.534560
Cost after iteration 2425: 0.534399
Cost after iteration 2430: 0.534239
Cost after iteration 2435: 0.534079
Cost after iteration 2440: 0.533920
Cost after iteration 2445: 0.533760
Cost after iteration 2450: 0.533602
Cost after iteration 2455: 0.533443
Cost after iteration 2460: 0.533285
Cost after iteration 2465: 0.533128
Cost after iteration 2470: 0.532970
Cost after iteration 2475: 0.532814
Cost after iteration 2480: 0.532657
Cost after iteration 2485: 0.532501
Cost after iteration 2490: 0.532345
Cost after iteration 2495: 0.532190
Cost after iteration 2500: 0.532035
Cost after iteration 2505: 0

In [23]:
def prediction(W1,W2,b1,b2,x_train):
    A2, cache = forward_propagation(W1,x_train,b1,W2,b2)
    predictions = np.round(A2)
    
    return predictions

In [24]:
predictions =prediction(W1,W2,b1,b2,x_train)
print ('Accuracy Train: %d' % float((np.dot(y_train, predictions.T) + np.dot(1 - y_train, 1 - predictions.T))/float(y_train.size)*100) + '%')
predictions = prediction(W1,W2,b1,b2,x_test)
print ('Accuracy Test: %d' % float((np.dot(y_test, predictions.T) + np.dot(1 - y_test, 1 - predictions.T))/float(y_test.size)*100) + '%')

Accuracy Train: 76%
Accuracy Test: 76%
